In [1]:
# getMe : bot에 대한 정보를 얻는다. 봇의 동작 확인
import requests
import json

API_TOKEN = ''
url = "https://api.telegram.org/bot%s/getme" % API_TOKEN

r = requests.get(url)
result = json.loads(r.text)
result

{'ok': True,
 'result': {'id': 872703060,
  'is_bot': True,
  'first_name': 'FinanceData_bot',
  'username': 'financeAlert_bot'}}

In [3]:
#getUpdates :봇 URL ( https://telegram.me/financeAlert_bot )에 접속하여 /start 명령을 보낸다.
# getMe : bot에 대한 정보를 얻는다. 봇의 동작 확인

url = "https://api.telegram.org/bot%s/getUpdates" % API_TOKEN

r = requests.get(url)
result = json.loads(r.text)
result

{'ok': True,
 'result': [{'update_id': 246036426,
   'message': {'message_id': 135,
    'from': {'id': 868118183,
     'is_bot': False,
     'first_name': '호성',
     'last_name': '차',
     'language_code': 'ko'},
    'chat': {'id': 868118183,
     'first_name': '호성',
     'last_name': '차',
     'type': 'private'},
    'date': 1564031038,
    'text': '/start',
    'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}]}

In [4]:
chat_id = ""
chat_text = "안녕하세요? financeAlert_bot 챗봇입니다"

url = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s" % (API_TOKEN, chat_id,  chat_text)
r = requests.get(url)
result = json.loads(r.text)
result

{'ok': True,
 'result': {'message_id': 136,
  'from': {'id': 872703060,
   'is_bot': True,
   'first_name': 'FinanceData_bot',
   'username': 'financeAlert_bot'},
  'chat': {'id': 868118183,
   'first_name': '호성',
   'last_name': '차',
   'type': 'private'},
  'date': 1564031082,
  'text': '안녕하세요? financeAlert_bot 챗봇입니다'}}

In [5]:
#특정 날짜의 DART 보고서 전체 가져오기

import json
import requests
import pandas as pd
from datetime import datetime, timedelta
from pandas.io.json import json_normalize

# 특정 날짜의 보고서 전체 가져오기
'''
get_dart_report_day
* date(str): 'yyyymmdd' (지정하지 않으면 오늘)
'''

def get_dart_report_day(date=None):
    auth = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
    headers={'Referer':'https://dart.fss.or.kr/dsap001/guide.do'}

    if date == None:
        date = datetime.today().strftime('%Y%m%d')
        
    url_tmpl = 'http://dart.fss.or.kr/api/search.json?'\
               'page_set=100&auth={auth}&start_dt={start_dt}&end_dt={end_dt}&page_no={page_no}'
    url = url_tmpl.format(auth=auth, start_dt=date, end_dt=date, page_no=1)
    r = requests.get(url, headers=headers)
    jo = json.loads(r.text)
    df = json_normalize(jo, 'list')
    
    for page in range(2, jo['total_page']+1):
        url = url_tmpl.format(auth=auth, start_dt=date, end_dt=date, page_no=page)
        r = requests.get(url, headers=headers)
        jo = json.loads(r.text)
        df = df.append(json_normalize(jo, 'list'))
        
    cols = {'crp_cd':'종목코드','crp_cls':'법인구분','crp_nm':'종목명','flr_nm':'제출인',
            'rcp_dt':'접수날짜','rcp_no':'접수번호','rmk':'비고','rpt_nm':'보고서명'}
    
    df.rename(columns=cols, inplace=True)
    
    if len(df) == 0:
        return df
      
    df['접수날짜'] = pd.to_datetime(df['접수날짜'])
    df.set_index('접수날짜', inplace=True)
    return df


In [28]:
pdf_link_tmpl = "http://dart.fss.or.kr/pdf/download/pdf.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
ifrs_link_tmpl = "http://dart.fss.or.kr/pdf/download/ifrs.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"


def get_report_attach_urls(rcp_no):
    attach_urls = []

    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)

    start_str = "javascript: viewDoc\('" + rcp_no + "', '"
    end_str = "', null, null, null,"
    reg = re.compile(start_str + '(.*)' + end_str)
    m = reg.findall(r.text)
    dcm_no = m[0]
    print(dcm_no)
    
    attach_urls.append(pdf_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    attach_urls.append(excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    attach_urls.append(ifrs_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    return attach_urls

In [32]:
df = get_dart_report_day()
len(df)
df
df.tail()


,종목코드,법인구분,종목명,제출인,접수번호,비고,보고서명
접수날짜,,,,,,,
2019-07-25,035420,Y,NAVER,NAVER,20190725800007,유,영업(잠정)실적(공정공시)
2019-07-25,035420,Y,NAVER,NAVER,20190725800005,유,연결재무제표기준영업(잠정)실적(공정공시)
2019-07-25,022100,K,포스코 ICT,포스코 ICT,20190725900002,코,[기재정정]단일판매ㆍ공급계약체결(자율공시)
2019-07-25,022100,K,포스코 ICT,포스코 ICT,20190725900001,코,[기재정정]단일판매ㆍ공급계약체결
2019-07-25,00908012,E,농협경제지주,농협경제지주,20190724000568,공,특수관계인으로부터기타유가증권매수


In [34]:
import re

keyword = ['1분기','2분기','3분기','사업보고서']

API_TOKEN = ''
chat_id = ""

for ix, row in df.iterrows():
    
    for j in keyword:
        
        if j in row['보고서명']:
            link = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['접수번호']
            chat_text = "[%s] %s %s" % (row['종목명'], row['보고서명'], link)
            url_tmpl = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s"
            url =  url_tmpl % (API_TOKEN, chat_id, chat_text)
            r = requests.get(url)

            file_name = row['종목명']+'_'+j+row['접수번호']
            print(file_name)
            print(get_report_attach_urls(row['접수번호']))
            
            local_download_url = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20170515003806&dcm_no=5653406&lang=ko'

            df = pd.read_excel(url, sheet_name='기본정보', index_col=0, skiprows=7)
        
        
#         result = json.loads(r.text)
#         print(result)

신보2018제2차유동화전문유한회사_사업보고서20190725000214
6819293
['http://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20190725000214&dcm_no=6819293', 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20190725000214&dcm_no=6819293&lang=ko', 'http://dart.fss.or.kr/pdf/download/ifrs.do?rcp_no=20190725000214&dcm_no=6819293&lang=ko']


UnicodeEncodeError: 'ascii' codec can't encode characters in position 90-91: ordinal not in range(128)

In [ ]:
def get_report_attach_urls(rcp_no):
    attach_urls = []

    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)

    start_str = "javascript: viewDoc\('" + rcp_no + "', '"
    end_str = "', null, null, null,"
    reg = re.compile(start_str + '(.*)' + end_str)
    m = reg.findall(r.text)
    dcm_no = m[0]
    
    attach_urls.append(pdf_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    attach_urls.append(excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    attach_urls.append(ifrs_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    return attach_urls